In [0]:
!pip install emojis
!pip install nltk
import emojis
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pyspark.sql import SparkSession
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import  *
from pyspark.sql import DataFrame
from collections import Counter
from typing import Iterable
import matplotlib.pyplot as plt
import seaborn as sns

Requirement already satisfied: emojis in /databricks/python3/lib/python3.7/site-packages (0.6.0)
 WARNING: You are using pip version 20.0.2; however, version 21.0.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command. 
Requirement already satisfied: nltk in /databricks/python3/lib/python3.7/site-packages (3.5)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.7/site-packages (from nltk) (0.14.1)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.7/site-packages (from nltk) (4.56.0)
Requirement already satisfied: regex in /databricks/python3/lib/python3.7/site-packages (from nltk) (2020.11.13)
Requirement already satisfied: click in /databricks/python3/lib/python3.7/site-packages (from nltk) (7.1.2)
 WARNING: You are using pip version 20.0.2; however, version 21.0.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command. 
[nltk_data] Downloading package stopwords to /home/root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!

In [0]:
# initialise sparkContext
spark = SparkSession.builder \
        .master('local[*]') \
        .appName('Venmo') \
        .config('spark.executor.memory', '10gb') \
        .config("spark.cores.max", "8") \
        .config("spark.executor.cores", "6") \
        .getOrCreate()

In [0]:
spark.sparkContext.getConf().getAll()

Out[8]: [('spark.files.useFetchCache', 'false'),
 ('spark.databricks.preemption.enabled', 'true'),
 ('spark.driver.tempDirectory', '/local_disk0/tmp'),
 ('spark.hadoop.fs.adl.impl.disable.cache', 'true'),
 ('spark.hadoop.parquet.block.size.row.check.max', '10'),
 ('spark.hadoop.fs.s3a.connection.maximum', '200'),
 ('spark.databricks.clusterUsageTags.numPerClusterInitScriptsV2', '0'),
 ('spark.hadoop.fs.s3a.fast.upload.active.blocks', '32'),
 ('spark.shuffle.reduceLocality.enabled', 'false'),
 ('spark.sql.streaming.checkpointFileManagerClass',
 'com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager'),
 ('spark.databricks.service.dbutils.repl.backend',
 'com.databricks.dbconnect.ReplDBUtils'),
 ('spark.databricks.clusterUsageTags.driverNodeType', 'dev-tier-node'),
 ('spark.hadoop.spark.sql.sources.outputCommitterClass',
 'com.databricks.backend.daemon.data.client.MapReduceDirectOutputCommitter'),
 ('spark.hadoop.fs.AbstractFileSystem.gs.impl',
 'shaded.databricks.V2_1_4.com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS'),
 ('spark.databricks.clusterUsageTags.instanceBootstrapType', 'ssh'),
 ('spark.streaming.driver.writeAheadLog.allowBatching', 'true'),
 ('spark.databricks.clusterSource', 'UI'),
 ('spark.hadoop.hive.server2.transport.mode', 'http'),
 ('spark.hadoop.hive.server2.thrift.http.cookie.auth.enabled', 'false'),
 ('spark.executor.memory', '8278m'),
 ('spark.databricks.driverNodeTypeId', 'dev-tier-node'),
 ('spark.sql.parquet.compression.codec', 'snappy'),
 ('spark.databricks.clusterUsageTags.hailEnabled', 'false'),
 ('spark.databricks.clusterUsageTags.clusterLogDeliveryEnabled', 'false'),
 ('spark.databricks.clusterUsageTags.containerType', 'LXC'),
 ('spark.eventLog.enabled', 'false'),
 ('spark.databricks.clusterUsageTags.isIMv2Enabled', 'false'),
 ('spark.databricks.clusterUsageTags.driverInstancePrivateIp',
 '10.172.225.140'),
 ('spark.hadoop.databricks.s3.create.deleteUnnecessaryFakeDirectories',
 'false'),
 ('spark.databricks.clusterUsageTags.driverContainerId',
 '331024f859054cbbb2e3d60cd2fb9546'),
 ('spark.databricks.clusterUsageTags.driverContainerPrivateIp',
 '10.172.233.228'),
 ('spark.hadoop.fs.wasb.impl',
 'shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem'),
 ('spark.executor.tempDirectory', '/local_disk0/tmp'),
 ('spark.databricks.workerNodeTypeId', 'dev-tier-node'),
 ('spark.hadoop.mapred.output.committer.class',
 'com.databricks.backend.daemon.data.client.DirectOutputCommitter'),
 ('spark.hadoop.hive.server2.thrift.http.port', '10000'),
 ('spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version', '2'),
 ('spark.sql.allowMultipleContexts', 'false'),
 ('spark.databricks.clusterUsageTags.clusterEbsVolumeSize', '0'),
 ('spark.home', '/databricks/spark'),
 ('spark.databricks.clusterUsageTags.clusterTargetWorkers', '0'),
 ('spark.sql.warehouse.dir', '/user/hive/warehouse'),
 ('spark.hadoop.hive.server2.idle.operation.timeout', '7200000'),
 ('spark.task.reaper.enabled', 'true'),
 ('spark.databricks.passthrough.s3a.tokenProviderClassName',
 'com.databricks.backend.daemon.driver.aws.AwsCredentialContextTokenProvider'),
 ('spark.storage.memoryFraction', '0.5'),
 ('spark.databricks.session.share', 'false'),
 ('spark.databricks.clusterUsageTags.driverInstanceId', 'i-0f0e0e66048d7bceb'),
 ('spark.databricks.clusterUsageTags.clusterResourceClass', 'default'),
 ('spark.databricks.clusterUsageTags.clusterFirstOnDemand', '0'),
 ('spark.driver.maxResultSize', '4g'),
 ('spark.repl.class.outputDir',
 '/local_disk0/tmp/repl/spark-8688220486511890135-25893c6e-ed2e-4916-8089-ef3192b180ce'),
 ('spark.databricks.delta.multiClusterWrites.enabled', 'true'),
 ('spark.databricks.clusterUsageTags.clusterSku', 'STANDARD_SKU'),
 ('spark.worker.cleanup.enabled', 'false'),
 ('spark.hadoop.fs.gs.impl.disable.cache', 'true'),
 ('spark.executor.extraJavaOptions',
 '-Djava.io.tmpdir=/local_disk0/tmp -XX:ReservedCodeCacheSize=256m -XX:+UseCodeCacheFlushing -Djava.security.properties=/databricks/spark/dbconf/java/

In [0]:
print(int(spark.sparkContext.getConf().get('spark.executor.cores','1')))
print(spark._jsc.sc().getExecutorMemoryStatus().size())
print(len(spark.sparkContext._jsc.sc().statusTracker().getExecutorInfos()))

1
1
1

In [0]:
data = spark.read.csv("/FileStore/tables/All_files.csv", header = 'TRUE')
data.cache()   # cache to speed up following operations
display(data)

user1,user2,transaction_type,datetime,app_description,description,story_id
2482900494712832556,2082497001160704615,payment,2018-08-07T02:11:16,Venmo for iPhone,fuk ya,2540405007614738995
2363395470786560486,2538731244355584742,payment,2018-08-07T02:11:16,Venmo for iPhone,:venmo_dollar:,2540405007799288777
2278060275531776951,1921315569139712500,payment,2018-08-07T02:11:15,Venmo for Android,🎉,2540404999133856612
2085027231825920983,2019594260709376161,payment,2018-08-07T02:11:15,Venmo for iPhone,Boyz,2540405000417313606
2029336009900032272,2080895330680832565,payment,2018-08-07T02:11:15,Venmo for iPhone,🥩,2540404998697648626
2302008090427392306,2228763010007040088,payment,2018-08-07T02:11:16,Venmo for iPhone,Internet,2540405008998859092
1843968157417472010,1863540549877760447,payment,2018-08-07T02:11:15,Venmo for iPhone,🤗,2540405002774512083
2127116921470976995,1406206291738624764,payment,2018-08-07T02:11:15,Venmo for iPhone,🚕🚕🚕,2540405002589962925
2354110858788864178,2423820619087872767,payment,2018-08-07T02:11:16,Venmo for iPhone,🤬,2540405008562652115
2308452848238592156,2467761313808384500,payment,2018-08-07T02:11:14,Venmo for Android,seltzer,2540404994536898980


In [0]:
# increase partition to speed up
data = data.repartition(200)
data.rdd.getNumPartitions()

Out[11]: 200

In [0]:
data.printSchema()

root
-- _c0: string (nullable = true)
-- _c1: string (nullable = true)
-- _c2: string (nullable = true)
-- _c3: string (nullable = true)
-- _c4: string (nullable = true)
-- _c5: string (nullable = true)
-- _c6: string (nullable = true)

In [0]:
data.count()

Out[13]: 7042335

In [0]:
text_dic = spark.read.format("csv") \
  .option("header", "true")  \
  .option("sep", ",") \
  .load("/FileStore/tables/Word_Classification_Dict.csv")
display(text_dic)

In [0]:
emoji_dic = spark.read.format("csv") \
  .option("header", "true")  \
  .option("sep", ",") \
  .load("/FileStore/tables/Emoji_Classification_Dictionary.csv")
display(emoji_dic)

In [0]:
# Convert columns to list
list(text_dic.select('People').dropna(how='all').toPandas()['People'])
list(text_dic.select('Food').dropna(how='all').toPandas()['Food'])
list(text_dic.select('Event').dropna(how='all').toPandas()['Event'])
list(text_dic.select('Activity').dropna(how='all').toPandas()['Activity'])
list(text_dic.select('Travel').dropna(how='all').toPandas()['Travel'])
list(text_dic.select('Transportation').dropna(how='all').toPandas()['Transportation'])
list(text_dic.select('Utility').dropna(how='all').toPandas()['Utility'])
list(text_dic.select('Cash').dropna(how='all').toPandas()['Cash'])
list(text_dic.select('Illegal/Sarcasm').dropna(how='all').toPandas()['Illegal/Sarcasm'])

In [0]:
list(emoji_dic.select('People').dropna(how='all').toPandas()['People'])
list(emoji_dic.select('Food').dropna(how='all').toPandas()['Food'])
list(emoji_dic.select('Event').dropna(how='all').toPandas()['Event'])
list(emoji_dic.select('Activity').dropna(how='all').toPandas()['Activity'])
list(emoji_dic.select('Travel').dropna(how='all').toPandas()['Travel'])
list(emoji_dic.select('Transportation').dropna(how='all').toPandas()['Transportation'])
list(emoji_dic.select('Utility').dropna(how='all').toPandas()['Utility'])
list(emoji_dic.select('Cash').dropna(how='all').toPandas()['Cash'])
list(emoji_dic.select('Illegal/Sarcasm').dropna(how='all').toPandas()['Illegal/Sarcasm'])

In [0]:
# add index column and create temporary view
data = data.sort(col("user1").desc()).withColumn("index", monotonically_increasing_id())
data.createOrReplaceTempView("Venmo")
data.show()

In [0]:
# Using flatMap() on Spark DataFrame - https://sparkbyexamples.com/spark/spark-flatmap-usage-with-example/
people = text_dic.select('People ').rdd.flatMap(lambda x: x).collect()
food = text_dic.select('Food ').rdd.flatMap(lambda x: x).collect()
event = text_dic.select('Event ').rdd.flatMap(lambda x: x).collect()
activity = text_dic.select('Activity').rdd.flatMap(lambda x: x).collect()
travel = text_dic.select('Travel ').rdd.flatMap(lambda x: x).collect()
transportation = text_dic.select('Transportation ').rdd.flatMap(lambda x: x).collect()
utility = text_dic.select('Utility ').rdd.flatMap(lambda x: x).collect()
cash = text_dic.select('Cash').rdd.flatMap(lambda x: x).collect()
illegal = text_dic.select('Illegal/Sarcasm ').rdd.flatMap(lambda x: x).collect()

In [0]:
# Using flatMap() on Spark DataFrame - https://sparkbyexamples.com/spark/spark-flatmap-usage-with-example/
people_emoji = emoji_dic.select('People').rdd.flatMap(lambda x: x).collect()
food_emoji = emoji_dic.select('Food').rdd.flatMap(lambda x: x).collect()
event_emoji = emoji_dic.select('Event').rdd.flatMap(lambda x: x).collect()
activity_emoji = emoji_dic.select('Activity').rdd.flatMap(lambda x: x).collect()
travel_emoji = emoji_dic.select('Travel').rdd.flatMap(lambda x: x).collect()
transportation_emoji = emoji_dic.select('Transportation').rdd.flatMap(lambda x: x).collect()
utility_emoji = emoji_dic.select('Utility').rdd.flatMap(lambda x: x).collect()

In [0]:
# use the emojis package instead of the provided dictionary, not sure if it's allowed, need to check with professor
@udf
def convert_emojis(text):
    text = " ".join(emojis.decode(text).replace(":", " ").replace("_", "").split())
    return text

@udf
def rm_punctuation(text):
  return text.translate(str.maketrans("","", string.punctuation))

@udf
def lemmatize(text):
    lemmatized_array = []
    lemmatizer = WordNetLemmatizer()
    for word in text:
      lemmatized_array.append(lemmatizer.lemmatize(word))
    return lemmatized_array
lemmatize_udf = udf(lemmatize, ArrayType(StringType()))

In [0]:
data = data.withColumn("replace_emoji", convert_emojis(col("description")))
data = data.withColumn("rm_punctuation", rm_punctuation(col("replace_emoji")))
tokenizer = Tokenizer(inputCol="rm_punctuation", outputCol="tokenized")
data = tokenizer.transform(data)
remover = StopWordsRemover(inputCol="tokenized", outputCol="rm_stopwords")
data = remover.transform(data)
data = data.withColumn("text_preprocessed", lemmatize_udf(col("rm_stopwords")))

columns_to_drop = ['replace_emoji', 'tokenized', 'rm_stopwords', "rm_punctuation"]
data = data.drop(*columns_to_drop)
display(data)

In [0]:
# use array to store the categories for each description, then get the most common one
@udf
def classification(text):
  categories = []
  for word in text:
    if word in people:
      categories.append('People')
    if word in food:
      categories.append('Food')
    if word in event:
      categories.append('Event')
    if word in activity:
      categories.append('Activity')
    if word in travel:
      categories.append('Travel')
    if word in transportation:
      categories.append('Transportation')
    if word in utility:
      categories.append('Utility')
    if word in cash:
      categories.append('Cash')
    if word in illegal:
      categories.append('Illegal')
  if not categories:
    return "Not Classified"
  else:
    return Counter(categories).most_common(1)[0][0]

In [0]:
data = data.withColumn("classification", classification(col("text_preprocessed")))
display(data)

In [0]:
@udf
def emoji_only(text):
  if len(text) == emojis.count(text):
    return 1
  return 0

data = data.withColumn("emoji_only", emoji_only(col("description")))
display(data)

In [0]:
# Percantage of transactions are emoji only
percent_emoji_only = data.select(sum("emoji_only")/data.count()).show()
percent_emoji_only

In [0]:
def get_emoji(text):
  if emojis.get(text):
    return list(emojis.get(text))
get_emoji_udf = udf(get_emoji, ArrayType(StringType()))

data = data.withColumn("emojis", get_emoji_udf(col("description")))
display(data)

In [0]:
bag_of_emojis = data.filter(col("emojis").isNotNull()).select("emojis").rdd.flatMap(lambda x: x).collect()
bag_of_emojis_flatten = []
for item in bag_of_emojis:
  for emoji in item:
    bag_of_emojis_flatten.append(emoji)
bag_of_emojis_flatten

In [0]:
# the top 5 most popular emoji:
Counter(bag_of_emojis_flatten).most_common(5)

In [0]:
def get_emoji_category(emoji):
  if emoji in people_emoji:
    return 'People'
  elif emoji in food_emoji:
    return 'Food'
  elif emoji in event_emoji:
    return 'Event'
  elif emoji in activity_emoji:
    return 'Activity'
  elif emoji in travel_emoji:
    return 'Travel'
  elif emoji in transportation_emoji:
    return 'Transportation'
  elif emoji in utility_emoji:
    return 'Utility'
  else:
    return 'Not Classified'

# use dictionary to speed up the running time
emoji_category_dic = dict()
emoji_category_arr = []

for emoji in bag_of_emojis_flatten:
  if emoji not in emoji_category_dic:
    emoji_category_dic[emoji] = get_emoji_category(emoji)
    emoji_category_arr.append(emoji_category_dic[emoji])
  else:
    emoji_category_arr.append(emoji_category_dic[emoji])

emoji_category_arr = [x for x in emoji_category_arr if x != "Not Classified"]
emoji_category_arr

In [0]:
# the top three most popular emoji categories are Food, People, Activity
Counter(emoji_category_arr).most_common(3)

In [0]:
data.createOrReplaceTempView("datatable")

In [0]:
%sql
-- merge user1 and user2
CREATE TABLE union_table
USING HIVE
AS 
  SELECT user1 AS user, classification
  FROM datatable
  UNION ALL
  SELECT user2 AS user, classification
  FROM datatable

In [0]:
# assumping the denominator doesn't include transactions that can't be classified
spend_profile_df = sqlContext.sql(
  '''
  SELECT user, CONCAT_WS(", ", 
                    CASE WHEN count_activity != 0 THEN CONCAT(ROUND(count_activity * 100), "% Activity") ELSE NULL END, 
                    CASE WHEN count_people != 0 THEN CONCAT(ROUND(count_people * 100), "% People") ELSE NULL END, 
                    CASE WHEN count_event != 0 THEN CONCAT(ROUND(count_event * 100), "% Event") ELSE NULL END, 
                    CASE WHEN count_travel != 0 THEN CONCAT(ROUND(count_travel * 100), "% Travel") ELSE NULL END, 
                    CASE WHEN count_transportation != 0 THEN CONCAT(ROUND(count_transportation * 100), "% Transportation") ELSE NULL END, 
                    CASE WHEN count_food != 0 THEN CONCAT(ROUND(count_food * 100), "% Food") ELSE NULL END, 
                    CASE WHEN count_utility != 0 THEN CONCAT(ROUND(count_utility * 100), "% Utility") ELSE NULL END, 
                    CASE WHEN count_cash != 0 THEN CONCAT(ROUND(count_cash * 100), "% Cash") ELSE NULL END,
                    CASE WHEN count_illegal != 0 THEN CONCAT(ROUND(count_illegal * 100), "% Illegal") ELSE NULL END
              ) AS spent_profile
  FROM
  (
    SELECT user,
        SUM(CASE WHEN classification == "Activity" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_activity,
        SUM(CASE WHEN classification == "Food" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_food,
        SUM(CASE WHEN classification == "People" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_people,
        SUM(CASE WHEN classification == "Event" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_event,
        SUM(CASE WHEN classification == "Travel" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_travel,
        SUM(CASE WHEN classification == "Transportation" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_transportation,
        SUM(CASE WHEN classification == "Utility" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_utility,
        SUM(CASE WHEN classification == "Cash" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_cash,
        SUM(CASE WHEN classification == "Illegal" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_illegal
    FROM union_table
    GROUP BY user
  )
  '''
)
display(spend_profile_df)

In [0]:
# merge user1 & user2
union_table_with_date_df = sqlContext.sql(
  '''
  SELECT user1 AS user, classification, datetime
  FROM datatable
  UNION ALL
  SELECT user2 AS user, classification, datetime
  FROM datatable
  '''
)

In [0]:
# read from storage
union_table_with_date_df = spark.read.format('csv').options(header='true', inferSchema='true')\
    .load('dbfs:/FileStore/union_table_with_date_df.csv')
union_table_with_date_df.createOrReplaceTempView("uniontable")

# find the first 12 life points
# assumption: 30 days a month
union_table_label_lifepoints = sqlContext.sql(
  '''
  SELECT user, classification, life_point
  FROM
  (
    SELECT user, classification,
      CASE WHEN datetime = first_day THEN 0
           WHEN DATEDIFF(datetime, first_day) <= 30 THEN 1
           WHEN DATEDIFF(datetime, first_day) <= 60 THEN 2
           WHEN DATEDIFF(datetime, first_day) <= 90 THEN 3
           WHEN DATEDIFF(datetime, first_day) <= 120 THEN 4
           WHEN DATEDIFF(datetime, first_day) <= 150 THEN 5
           WHEN DATEDIFF(datetime, first_day) <= 180 THEN 6
           WHEN DATEDIFF(datetime, first_day) <= 210 THEN 7
           WHEN DATEDIFF(datetime, first_day) <= 240 THEN 8
           WHEN DATEDIFF(datetime, first_day) <= 270 THEN 9
           WHEN DATEDIFF(datetime, first_day) <= 300 THEN 10
           WHEN DATEDIFF(datetime, first_day) <= 330 THEN 11
           WHEN DATEDIFF(datetime, first_day) <= 360 THEN 12
      END AS life_point
    FROM 
    (
      SELECT *, MIN(datetime) OVER (PARTITION BY user) AS first_day
      FROM uniontable
    )
  )
  WHERE life_point <= 12
  '''
)
display(union_table_label_lifepoints)

In [0]:
# read from storage
union_table_qualified_subset = spark.read.format('csv').options(header='true', inferSchema='true')\
    .load('dbfs:/FileStore/union_table_label_lifepoints.csv')
union_table_label_lifepoints.createOrReplaceTempView("union_table_subset")

%sql
--  count classifications grouping by user, life_point
--  assumping the denominator doesn't include transactions that can't be classified
DROP TABLE IF EXISTS classfication_by_time;

CREATE TABLE classfication_by_time
USING HIVE
AS 
  SELECT user, life_point,
    SUM(CASE WHEN classification == "Activity" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_activity,
    SUM(CASE WHEN classification == "Food" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_food,
    SUM(CASE WHEN classification == "People" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_people,
    SUM(CASE WHEN classification == "Event" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_event,
    SUM(CASE WHEN classification == "Travel" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_travel,
    SUM(CASE WHEN classification == "Transportation" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_transportation,
    SUM(CASE WHEN classification == "Utility" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_utility,
    SUM(CASE WHEN classification == "Cash" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_cash,
    SUM(CASE WHEN classification == "Illegal" THEN 1 ELSE 0 END)/SUM(CASE WHEN classification != "Not Classified" THEN 1 ELSE 0 END) AS count_illegal
  FROM union_table_subset
  GROUP BY user, life_point;
    
SELECT * FROM classfication_by_time

In [0]:
# create profile by user, life_point
dynamic_profile = sqlContext.sql(
  '''
  SELECT user, life_point,
             CONCAT_WS(", ", 
                    CASE WHEN count_activity != 0 THEN CONCAT(ROUND(count_activity * 100), "% Activity") ELSE NULL END, 
                    CASE WHEN count_people != 0 THEN CONCAT(ROUND(count_people * 100), "% People") ELSE NULL END, 
                    CASE WHEN count_event != 0 THEN CONCAT(ROUND(count_event * 100), "% Event") ELSE NULL END, 
                    CASE WHEN count_travel != 0 THEN CONCAT(ROUND(count_travel * 100), "% Travel") ELSE NULL END, 
                    CASE WHEN count_transportation != 0 THEN CONCAT(ROUND(count_transportation * 100), "% Transportation") ELSE NULL END, 
                    CASE WHEN count_food != 0 THEN CONCAT(ROUND(count_food * 100), "% Food") ELSE NULL END, 
                    CASE WHEN count_utility != 0 THEN CONCAT(ROUND(count_utility * 100), "% Utility") ELSE NULL END, 
                    CASE WHEN count_cash != 0 THEN CONCAT(ROUND(count_cash * 100), "% Cash") ELSE NULL END,
                    CASE WHEN count_illegal != 0 THEN CONCAT(ROUND(count_illegal * 100), "% Illegal") ELSE NULL END
              ) AS spent_profile
  FROM classfication_by_time
  ORDER BY user, life_point
  '''
)
display(dynamic_profile)